# BEA API Call  

In [1]:
import requests
import pandas as pd
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
def percent(x, y):
    return (x/y)*100
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
GNRC = ['47161', #Stewart
       '47125', #Montgomery
       '47083', #Houston
       '47085', #Humphreys
       '47043', #Dickson
       '47021', #Cheatham
       '47147', #Robertson
       '47165', #Sumner
       '47037', #Davidson
       '47189', #Wilson
       '47169', #Trousdale
       '47187', #Williamson
       '47149', #Rutherford
       '47119'] #Maury

In [3]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)
#create a variable that contains your api key
census_key = keys_dict_2['CENSUS']
bea_key = keys_dict_2['BEA']

In [6]:
url_str = 'https://apps.bea.gov/api/data/?&UserID={}'.format(bea_key)
method = '&method=GetData'
dataset = '&datasetname=Regional'
table = '&TableName=SQINC1'
linecode = '&LineCode=ALL'
year = '&Year=2020'

In [9]:
data_appended = []
for x in GNRC:
    geo = '&GeoFips={}'.format(x)
    url = '{}{}{}{}{}{}{}'.format(url_str, method, dataset, table, linecode, geo, year)
    r = requests.get(url).json()
    data = pd.DataFrame(r['BEAAPI']['Results'])#['Data']['Thing'])
    data_appended.append(data)
data_appended = pd.concat(data_appended).reset_index(drop = True)

In [10]:
data_appended.head(100)

,Error
0,101
1,Unknown error.
2,{'ParameterInput': [{'ParameterName': 'TableNa...
3,101
4,Unknown error.
5,{'ParameterInput': [{'ParameterName': 'TableNa...
6,101
7,Unknown error.
8,{'ParameterInput': [{'ParameterName': 'TableNa...
9,101


In [20]:
data_filtered = data_appended.loc[(data_appended['Description'] == 'Total employment (number of jobs)')|
                                  (data_appended['Description'] == 'Farm employment')|
                                  (data_appended['Description'] == 'Forestry, fishing, and related activities')|
                                  (data_appended['Description'] == 'Mining, quarrying, and oil and gas extraction')|
                                  (data_appended['Description'] == 'Utilities')|
                                  (data_appended['Description'] == 'Construction')|
                                  (data_appended['Description'] == 'Manufacturing')|
                                  (data_appended['Description'] == 'Wholesale trade')|
                                  (data_appended['Description'] == 'Retail trade')|
                                  (data_appended['Description'] == 'Transportation and warehousing')|
                                  (data_appended['Description'] == 'Information')|
                                  (data_appended['Description'] == 'Finance and insurance')|
                                  (data_appended['Description'] == 'Real estate and rental and leasing')|
                                  (data_appended['Description'] == 'Professional, scientific, and technical services')|
                                  (data_appended['Description'] == 'Management of companies and enterprises')|
                                  (data_appended['Description'] == 'Administrative and support and waste management and remediation services')|
                                  (data_appended['Description'] == 'Educational services')|
                                  (data_appended['Description'] == 'Health care and social assistance')|
                                  (data_appended['Description'] == 'Arts, entertainment, and recreation')|
                                  (data_appended['Description'] == 'Accommodation and food services')|
                                  (data_appended['Description'] == 'Other services (except government and government enterprises)')|
                                  (data_appended['Description'] == 'Government and government enterprises')|
                                  (data_appended['Description'] == 'State and local')].reset_index(drop = True)
data = data_filtered[['GeoName', 'Description', 'DataValue']]

**STOP** here to see what is suppressed across different counties.  

In [21]:
# allsuppressedvalues = data.loc[data['DataValue'] == '(D)']
# allsuppressedvalues

There are industries suppressed in more than one custom "sector" for the following counties: Stewart, Houston, Humphreys, Cheatham, and Trousdale.  


There are industries suppressed in only the "industry sector" for the following counties: Montgomery, Dickson, Robertson, Sumner, Wilson, Williamson, Rutherford, and Maury.  


I'm going to calculate the "industry" industry as a difference between that grouping and total employment for all counties *except* Stewart, Houston, Humphreys, Cheatham, and Trousdale.

In [22]:
suppressed = data.loc[(data['GeoName'] == 'Stewart, TN')|
                      (data['GeoName'] == 'Houston, TN')|
                      (data['GeoName'] == 'Humphreys, TN')|
                      (data['GeoName'] == 'Cheatham, TN')|
                      (data['GeoName'] == 'Trousdale, TN')].reset_index(drop = True)
notsuppressed = data.loc[(data['GeoName'] == 'Montgomery, TN')|
                         (data['GeoName'] == 'Dickson, TN')|
                         (data['GeoName'] == 'Robertson, TN')|
                         (data['GeoName'] == 'Sumner, TN')|
                         (data['GeoName'] == 'Wilson, TN')|
                         (data['GeoName'] == 'Williamson, TN')|
                         (data['GeoName'] == 'Rutherford, TN')|
                         (data['GeoName'] == 'Maury, TN')|
                         (data['GeoName'] == 'Davidson, TN')].reset_index(drop = True)
notsuppressed = notsuppressed.loc[(notsuppressed['Description'] != 'Forestry, fishing, and related activities')&
                                  (notsuppressed['Description'] != 'Mining, quarrying, and oil and gas extraction')&
                                  (notsuppressed['Description'] != 'Utilities')&
                                  (notsuppressed['Description'] != 'Construction')&
                                  (notsuppressed['Description'] != 'Manufacturing')&
                                  (notsuppressed['Description'] != 'Wholesale trade')&
                                  (notsuppressed['Description'] != 'Transportation and warehousing')].reset_index(drop = True)
notsuppressed = notsuppressed.set_index(['GeoName', 'Description'])
notsuppressed = notsuppressed.transpose()
#the datavalues are strings now, so we need to remove the commas and make them into float dtypes
cols = notsuppressed.columns
for column in cols:
    notsuppressed[column] = notsuppressed[column].str.replace(',', '')
notsuppressed[cols] = notsuppressed[cols].astype(float)
df = notsuppressed

In [23]:
#list of non suppressed counties to loop through for calculating 'industry' difference.
nonsupcounties = ['Montgomery, TN', 'Dickson, TN', 'Robertson, TN', 'Sumner, TN', 'Wilson, TN', 'Williamson, TN', 'Rutherford, TN', 'Maury, TN', 'Davidson, TN']

In [24]:
df.head()

GeoName                        Montgomery, TN                                                                                                    \
Description Total employment (number of jobs) Farm employment Retail trade Information Finance and insurance Real estate and rental and leasing   
DataValue                             81903.0           879.0      11088.0      1037.0                2342.0                             4024.0   

GeoName                                                                                               \
Description Professional, scientific, and technical services Management of companies and enterprises   
DataValue                                             2860.0                                   438.0   

GeoName                                                                                                                                      \
Description Administrative and support and waste management and remediation services Educational services Health care and social assistance   
DataValue                                               5229.0                                     1443.0                            8597.0   

GeoName                                                                                                                                        \
Description Arts, entertainment, and recreation Accommodation and food services Other services (except government and government enterprises)   
DataValue                                1213.0                          8242.0                                             5670.0              

GeoName                                                                                 Dickson, TN                                           \
Description Government and government enterprises State and local Total employment (number of jobs) Farm employment Retail trade Information   
DataValue                                 12665.0         10338.0                           25651.0          1123.0       2985.0       139.0   

GeoName                                                                                                                \
Description Finance and insurance Real estate and rental and leasing Professional, scientific, and technical services   
DataValue                   916.0                              800.0                                            759.0   

GeoName                                                                                                                                            \
Description Management of companies and enterprises Administrative and support and waste management and remediation services Educational services   
DataValue                                      83.0                                             1806.0                                      185.0   

GeoName                                                                                                            \
Description Health care and social assistance Arts, entertainment, and recreation Accommodation and food services   
DataValue                              2908.0                               368.0                          1933.0   

GeoName                                                                                                                          \
Description Other services (except government and government enterprises) Government and government enterprises State and local   
DataValue                                               1497.0                                           2793.0          2541.0   

GeoName                         Robertson, TN                                                                                                    \
Description Total employment (number of jobs) Farm employment Retail trade Information Finance and insurance Real estate and rental and leasing   
DataValue                             34035.0          1511.0       3742.0       150.0        

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, DataValue to DataValue
Columns: 144 entries, ('Montgomery, TN', 'Total employment (number of jobs)') to ('Maury, TN', 'State and local')
dtypes: float64(144)
memory usage: 1.1+ KB


In [11]:
newstuff = {}
for county in nonsupcounties:
    newstuff[('{}'.format(county), 'Industry:Total Jobs')] = df[('{}'.format(county), 'Total employment (number of jobs)')]
    professional = [df[('{}'.format(county), 'Information')], df[('{}'.format(county), 'Finance and insurance')], 
                    df[('{}'.format(county), 'Real estate and rental and leasing')],
                    df[('{}'.format(county), 'Professional, scientific, and technical services')],
                    df[('{}'.format(county), 'Management of companies and enterprises')],
                    df[('{}'.format(county), 'Administrative and support and waste management and remediation services')], 
                    df[('{}'.format(county), 'Educational services')], df[('{}'.format(county), 'Health care and social assistance')]]
    newstuff[('{}'.format(county), 'Industry:Professional')] = sum(professional)
    newstuff[('{}'.format(county), 'Industry:Percent Professional')] = percent(newstuff[('{}'.format(county), 'Industry:Professional')], 
                                                                               df[('{}'.format(county), 'Total employment (number of jobs)')])
    retail = [df[('{}'.format(county), 'Retail trade')], df[('{}'.format(county), 'Arts, entertainment, and recreation')], 
              df[('{}'.format(county), 'Accommodation and food services')], 
              df[('{}'.format(county), 'Other services (except government and government enterprises)')]]
    newstuff[('{}'.format(county), 'Industry:Retail')] = sum(retail)
    newstuff[('{}'.format(county), 'Industry:Percent Retail')] = percent(newstuff[('{}'.format(county), 'Industry:Retail')], 
                                                                   df[('{}'.format(county), 'Total employment (number of jobs)')])
    newstuff[('{}'.format(county), 'Industry:Government')] = df[('{}'.format(county), 'Government and government enterprises')]
    newstuff[('{}'.format(county), 'Industry:Percent Government')] = percent(newstuff[('{}'.format(county), 'Industry:Government')], 
                                                                       df[('{}'.format(county), 'Total employment (number of jobs)')])
    newstuff[('{}'.format(county), 'Industry:Farm')] = df[('{}'.format(county), 'Farm employment')]
    newstuff[('{}'.format(county), 'Industry:Percent Farm')] = percent(newstuff[('{}'.format(county), 'Industry:Farm')], 
                                                                 df[('{}'.format(county), 'Total employment (number of jobs)')])
    besidesindustry = [newstuff[('{}'.format(county), 'Industry:Farm')], newstuff[('{}'.format(county), 'Industry:Government')], 
                       newstuff[('{}'.format(county), 'Industry:Retail')],newstuff[('{}'.format(county), 'Industry:Professional')]]
    newstuff[('{}'.format(county), 'Industry:Industry')] = df[('{}'.format(county), 'Total employment (number of jobs)')] - sum(besidesindustry)
    newstuff[('{}'.format(county), 'Industry:Percent Industry')] = percent(newstuff[('{}'.format(county), 'Industry:Industry')], 
                                                                     df[('{}'.format(county), 'Total employment (number of jobs)')])
    newdata = pd.concat(newstuff, axis = 1)

In [12]:
newdata.head()

Montgomery, TN                                                                                                                  \
          Industry:Total Jobs Industry:Professional Industry:Percent Professional Industry:Retail Industry:Percent Retail Industry:Government   
DataValue             81903.0               25970.0                      31.70824         26213.0               32.004933             12665.0   

                                                                                                                              Dickson, TN  \
          Industry:Percent Government Industry:Farm Industry:Percent Farm Industry:Industry Industry:Percent Industry Industry:Total Jobs   
DataValue                   15.463414         879.0              1.073221           16176.0                 19.750192             25651.0   

                                                                                                                           \
          Industry:Professional Industry:Percent Professional Industry:Retail Industry:Percent Retail Industry:Government   
DataValue                7596.0                     29.612881          6783.0               26.443414              2793.0   

                                                                                                                            Robertson, TN  \
          Industry:Percent Government Industry:Farm Industry:Percent Farm Industry:Industry Industry:Percent Industry Industry:Total Jobs   
DataValue                   10.888464        1123.0              4.377997            7356.0                 28.677245             34035.0   

                                                                                                                           \
          Industry:Professional Industry:Percent Professional Industry:Retail Industry:Percent Retail Industry:Government   
DataValue                7786.0                     22.876451          8196.0               24.081093              3900.0   

                                                                                                                               Sumner, TN  \
          Industry:Percent Government Industry:Farm Industry:Percent Farm Industry:Industry Industry:Percent Industry Industry:Total Jobs   
DataValue                   11.458792        1511.0              4.439548           12642.0                 37.144116             88436.0   

                                                                                                                           \
          Industry:Professional Industry:Percent Professional Industry:Retail Industry:Percent Retail Industry:Government   
DataValue               31224.0                     35.306889         23884.0               27.007101              9020.0   

                                                                                                                               Wilson, TN  \
          Industry:Percent Government Industry:Farm Industry:Percent Farm Industry:Industry Industry:Percent Industry Industry:Total Jobs   
DataValue                   10.199466        1443.0              1.631688           22865.0                 25.854855             76738.0   

                                                                                                                           \
          Industry:Professional Industry:Percent Professional Industry:Retail Industry:Percent Retail Industry:Government   
DataValue               22722.0                     29.609841         19934.0                 25.9767              6397.0   

                                                                                                                           Williamson, TN  \
          Industry:Percent Government Industry:Farm Industry:Percent Farm Industry:Industry Industry:Percent Industry Industry:Total Jobs   
DataValue                    8.336157        1462.0              1.905184           26223.0      

In [67]:
notsuppressed = newdata
notsuppressed.head()

Montgomery, TN                                                                                                                  \
          Industry:Total Jobs Industry:Professional Industry:Percent Professional Industry:Retail Industry:Percent Retail Industry:Government   
DataValue             81903.0               25970.0                      31.70824         26213.0               32.004933             12665.0   

                                                                                                                              Dickson, TN  \
          Industry:Percent Government Industry:Farm Industry:Percent Farm Industry:Industry Industry:Percent Industry Industry:Total Jobs   
DataValue                   15.463414         879.0              1.073221           16176.0                 19.750192             25651.0   

                                                                                                                           \
          Industry:Professional Industry:Percent Professional Industry:Retail Industry:Percent Retail Industry:Government   
DataValue                7596.0                     29.612881          6783.0               26.443414              2793.0   

                                                                                                                            Robertson, TN  \
          Industry:Percent Government Industry:Farm Industry:Percent Farm Industry:Industry Industry:Percent Industry Industry:Total Jobs   
DataValue                   10.888464        1123.0              4.377997            7356.0                 28.677245             34035.0   

                                                                                                                           \
          Industry:Professional Industry:Percent Professional Industry:Retail Industry:Percent Retail Industry:Government   
DataValue                7786.0                     22.876451          8196.0               24.081093              3900.0   

                                                                                                                               Sumner, TN  \
          Industry:Percent Government Industry:Farm Industry:Percent Farm Industry:Industry Industry:Percent Industry Industry:Total Jobs   
DataValue                   11.458792        1511.0              4.439548           12642.0                 37.144116             88436.0   

                                                                                                                           \
          Industry:Professional Industry:Percent Professional Industry:Retail Industry:Percent Retail Industry:Government   
DataValue               31224.0                     35.306889         23884.0               27.007101              9020.0   

                                                                                                                               Wilson, TN  \
          Industry:Percent Government Industry:Farm Industry:Percent Farm Industry:Industry Industry:Percent Industry Industry:Total Jobs   
DataValue                   10.199466        1443.0              1.631688           22865.0                 25.854855             76738.0   

                                                                                                                           \
          Industry:Professional Industry:Percent Professional Industry:Retail Industry:Percent Retail Industry:Government   
DataValue               22722.0                     29.609841         19934.0                 25.9767              6397.0   

                                                                                                                           Williamson, TN  \
          Industry:Percent Government Industry:Farm Industry:Percent Farm Industry:Industry Industry:Percent Industry Industry:Total Jobs   
DataValue                    8.336157        1462.0              1.905184           26223.0      

In [68]:
suppressed.head(40)

,GeoName,Description,DataValue
0,"Stewart, TN",Total employment (number of jobs),"4,664"
1,"Stewart, TN",Farm employment,354
2,"Stewart, TN","Forestry, fishing, and related activities",(D)
3,"Stewart, TN","Mining, quarrying, and oil and gas extraction",1
4,"Stewart, TN",Utilities,(D)
5,"Stewart, TN",Construction,(D)
6,"Stewart, TN",Manufacturing,702
7,"Stewart, TN",Wholesale trade,38
8,"Stewart, TN",Retail trade,(D)
9,"Stewart, TN",Transportation and warehousing,(D)


In [69]:
suppressed['DataValue'] = suppressed['DataValue'].str.replace(',', '')

Now we'll go through the other counties one at a time then append them.

In [72]:

stewart = suppressed.loc[suppressed['GeoName'] == 'Stewart, TN'].reset_index(drop = True)
stewart['DataValue'] = stewart['DataValue'].str.replace('\\(D\\)', '0')
stewart['DataValue'] = stewart['DataValue'].astype(float)

C:\Users\jmccall\AppData\Local\Temp/ipykernel_21036/852542346.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  stewart['DataValue'] = stewart['DataValue'].str.replace('\\(D\\)', '0')


In [74]:
humphreys = suppressed.loc[suppressed['GeoName'] == 'Humphreys, TN'].reset_index(drop = True)
humphreys

,GeoName,Description,DataValue
0,"Humphreys, TN",Total employment (number of jobs),8506
1,"Humphreys, TN",Farm employment,623
2,"Humphreys, TN","Forestry, fishing, and related activities",(D)
3,"Humphreys, TN","Mining, quarrying, and oil and gas extraction",29
4,"Humphreys, TN",Utilities,(D)
5,"Humphreys, TN",Construction,661
6,"Humphreys, TN",Manufacturing,1455
7,"Humphreys, TN",Wholesale trade,(D)
8,"Humphreys, TN",Retail trade,888
9,"Humphreys, TN",Transportation and warehousing,420


In [73]:
stewart.head(30)

,GeoName,Description,DataValue
0,"Stewart, TN",Total employment (number of jobs),4664.0
1,"Stewart, TN",Farm employment,354.0
2,"Stewart, TN","Forestry, fishing, and related activities",0.0
3,"Stewart, TN","Mining, quarrying, and oil and gas extraction",1.0
4,"Stewart, TN",Utilities,0.0
5,"Stewart, TN",Construction,0.0
6,"Stewart, TN",Manufacturing,702.0
7,"Stewart, TN",Wholesale trade,38.0
8,"Stewart, TN",Retail trade,0.0
9,"Stewart, TN",Transportation and warehousing,0.0


For Stewart, the only 0 is for management of companies and enterprises. This mean's it's missing:   
+ Forestry, fishing and related activities  
+ 
I'm going to replace the suppressed data with 0's and add them, but will note this in the export. I'm also going to replace the commas with nothing and convert the string dtypes to floats.

In [ ]:
county = 
newstuff[('{}'.format(county), 'Industry:Total Jobs')] = df[('{}'.format(county), 'Total employment (number of jobs)')]
professional = [df[('{}'.format(county), 'Information')], df[('{}'.format(county), 'Finance and insurance')], 
                df[('{}'.format(county), 'Real estate and rental and leasing')],
                df[('{}'.format(county), 'Professional, scientific, and technical services')],
                df[('{}'.format(county), 'Management of companies and enterprises')],
                df[('{}'.format(county), 'Administrative and support and waste management and remediation services')], 
                df[('{}'.format(county), 'Educational services')], df[('{}'.format(county), 'Health care and social assistance')]]
newstuff[('{}'.format(county), 'Industry:Professional')] = sum(professional)
newstuff[('{}'.format(county), 'Industry:Percent Professional')] = percent(newstuff[('{}'.format(county), 'Industry:Professional')], 
                                                                           df[('{}'.format(county), 'Total employment (number of jobs)')])
retail = [df[('{}'.format(county), 'Retail trade')], df[('{}'.format(county), 'Arts, entertainment, and recreation')], 
          df[('{}'.format(county), 'Accommodation and food services')], 
          df[('{}'.format(county), 'Other services (except government and government enterprises)')]]
newstuff[('{}'.format(county), 'Industry:Retail')] = sum(retail)
newstuff[('{}'.format(county), 'Industry:Percent Retail')] = percent(newstuff[('{}'.format(county), 'Industry:Retail')], 
                                                                     df[('{}'.format(county), 'Total employment (number of jobs)')])
newstuff[('{}'.format(county), 'Industry:Government')] = df[('{}'.format(county), 'Government and government enterprises')]
newstuff[('{}'.format(county), 'Industry:Percent Government')] = percent(newstuff[('{}'.format(county), 'Industry:Government')], 
                                                                         df[('{}'.format(county), 'Total employment (number of jobs)')])
newstuff[('{}'.format(county), 'Industry:Farm')] = df[('{}'.format(county), 'Farm employment')]
newstuff[('{}'.format(county), 'Industry:Percent Farm')] = percent(newstuff[('{}'.format(county), 'Industry:Farm')], 
                                                                   df[('{}'.format(county), 'Total employment (number of jobs)')])
besidesindustry = [newstuff[('{}'.format(county), 'Industry:Farm')], newstuff[('{}'.format(county), 'Industry:Government')], 
                   newstuff[('{}'.format(county), 'Industry:Retail')],newstuff[('{}'.format(county), 'Industry:Professional')]]
newstuff[('{}'.format(county), 'Industry:Industry')] = df[('{}'.format(county), 'Total employment (number of jobs)')] - sum(besidesindustry)
newstuff[('{}'.format(county), 'Industry:Percent Industry')] = percent(newstuff[('{}'.format(county), 'Industry:Industry')], 
                                                                       df[('{}'.format(county), 'Total employment (number of jobs)')])